In [273]:
import numpy as np
import pandas as pd
import re
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

In [274]:
df=pd.read_csv('fake_job_postings.csv')

In [275]:
#--------1. split the columns to country, state, and city

# to use country only?
df['country']=df.location.str.split(',',expand=True)[:][0]

df['state']=df.location.str.split(',',expand=True)[:][1]
df['city']=df.location.str.split(',',expand=True)[:][2]

In [276]:

#----------2. replacing different kinds of missing value to np.nan
df.state = df.state.str.strip()
df.state.fillna(value=np.nan, inplace=True)
df.state.replace('', np.nan, inplace=True)
df.state.replace(' ', np.nan, inplace=True)


#df['state'].sort_values().unique()

In [277]:
#----------3. clear the whitespaces and signs at the start/end

df.city = df.city.str.strip(' /:\\')

#---------4. replacing different kinds of missing value to np.nan
df.city.fillna(value=np.nan, inplace=True)
df.city.replace('', np.nan, inplace=True)
df.city.replace(' ', np.nan, inplace=True)
df['city']=df.city.str.lower()
#------

#print(df['city'].sort_values().unique().tolist())

In [278]:
df.salary_range.fillna(value=np.nan, inplace=True)
df.salary_range.replace('', np.nan, inplace=True)
df.salary_range.replace(' ', np.nan, inplace=True)

#---------4. spilt salary range into min and max

df['min_salary']=df.salary_range.str.split('-',expand=True)[:][0]
df['max_salary']=df.salary_range.str.split('-',expand=True)[:][1]

df.max_salary.fillna(value=np.nan, inplace=True)

In [279]:
#----------5. for entry of salary_range as date, max and min salary are grouped as null value
df.loc[df['max_salary'].isin(['Apr', 'Dec', 'Jun', 'Nov', 'Oct', 'Sep']),['max_salary', 'min_salary']]=np.nan
df.loc[df['min_salary'].isin(['Dec', 'Jun', 'Oct']),['max_salary', 'min_salary']]=np.nan

#convert them into numerical value
df[['min_salary','max_salary']] = df[['min_salary','max_salary']].astype(float)

#for regression model, need to impute NaN values to median/mean
# df['max_salary'].fillna(value=df['max_salary'].mean(), inplace=True)
# df['min_salary'].fillna(value=df['min_salary'].mean(), inplace=True)

In [ ]:
#-------------------end of processing for column salary_range and location

In [ ]:
#-------------------start cleaning for text columns

In [ ]:
# this function for preprocessing text is used linked from the next function

def preprocess_text(text):
    # Tokenise words while ignoring punctuation
    tokeniser = RegexpTokenizer(r'(?u)\b\w\w+\b')
    tokens = tokeniser.tokenize(text)
    
    # Lowercase and lemmatise 
    lemmatiser = WordNetLemmatizer()
    lemmas = [lemmatiser.lemmatize(token.lower(), pos='v') for token in tokens]
    
    # Remove stopwords
    keywords= [lemma for lemma in lemmas if lemma not in stopwords.words('english')]
    return keywords



In [ ]:
def vectorise_column(df, column):
    # Fill empty columns with "Unspecified"
    df[column] = df[column].fillna("Unspecified")
    
    # Create an instance of TfidfVectorizer
    vectoriser = TfidfVectorizer(analyzer=preprocess_text, ngram_range = (1,2))

    # Fit to the data and transform to feature matrix
    text_column = vectoriser.fit_transform(df[column])

    # Convert sparse matrix to dataframe
    text_column = pd.DataFrame.sparse.from_spmatrix(text_column)

    # Save mapping on which index refers to which words
    col_map = {v:k for k, v in vectoriser.vocabulary_.items()}

    # Rename each column using the mapping
    for col in text_column.columns:
        text_column.rename(columns={col: col_map[col]}, inplace=True)
    
    # Combined to dataframe
    combined = [df, text_column]
    df = pd.concat(combined, axis =1)
    
    return df

In [ ]:
# Create new table for text which can be merged to main table later

df_text = df.loc[:, ['company_profile', 'description', 'requirements', 'benefits']]
df_text['text'] = df_text.apply(lambda row: (str(row['company_profile']) + ' ' + str(row['description'])  
                                            + ' ' + str(row['requirements']) + ' ' + str(row['benefits'])), axis = 1)
to_merge = vectorise_column(df_text, 'text')

# Only the following columns need to be merged
# to_merge.iloc[:,4:]

In [ ]:
#-------------------end cleaning for text columns